tmp

In [20]:
import pandas as pd
import scipy.io.wavfile as wav
import numpy as np
from os import path

In [2]:
bpm = 120
bps = bpm / 60

In [3]:
folder = "wygibasy"
dir_to_song = path.join(folder, "song.txt")

In [4]:
dir_to_song

'wygibasy/song.txt'

In [5]:
song = pd.read_csv(dir_to_song, header = None, dtype = str)
song

,0
0,01
1,01
2,01
3,02
4,01
5,01
6,01
7,03


In [6]:
nrow_song = len(song)
path_to_track = [None] * nrow_song
for i in range(nrow_song):
    name_of_track = 'track' + song.loc[i, 0] + '.txt'
    path_to_track[i] = path.join(folder, name_of_track)

In [7]:
path_to_track

['wygibasy/track01.txt',
 'wygibasy/track01.txt',
 'wygibasy/track01.txt',
 'wygibasy/track02.txt',
 'wygibasy/track01.txt',
 'wygibasy/track01.txt',
 'wygibasy/track01.txt',
 'wygibasy/track03.txt']

In [8]:
#bierzemy pod lupe tylko pierwsza sciezke
track = pd.read_csv(path_to_track[0], sep = ' ', header = None, dtype = str)

In [9]:
#wszystkie sample mają taką samą częstotliwosć próbkowania
f = 44100

In [10]:
next_sample = f // bps

In [11]:
next_sample = int(next_sample) #tyle wierszy zajmie jedna linijka w track

In [12]:
next_sample

22050

In [14]:
nrow_track = track.shape[0]
nrow_track

16

In [15]:
ncol_track = track.shape[1]
ncol_track

4

In [17]:
N = next_sample * nrow_track #+'ostatki' zeby ich nie ucinac

In [21]:
track_all = np.zeros((N, ncol_track))

In [22]:
track_all

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [41]:
list_of_samples = list()
for k in range(ncol_track):
    column_sample = track.loc[:, k]
    #zakladam, ze w kolumnie tylko 1 ten sam sample
    column = np.zeros((N, 2))
    #sprawdzić, jaki numer
    num_sample = list(set(column_sample).difference(['--']))[0]
    sample = wav.read('wygibasy/sample' + num_sample + '.wav')
    n = sample[1].shape[0]
    if n > next_sample:
        #ubezpieczamy sie na wyrost, nawet jezeli sample nie bedzie na koncu bo bedzie --
        how_many_to_add = n - next_sample
        column = np.zeros((N + how_many_to_add, 2))
    for i, j in enumerate(column_sample):
        if j != '--':
            column[i * next_sample : i * next_sample + n, :] += sample[1]
            #tu też powinniśmy normalizować w sumie bo jak dodajemy duzo na siebie to moze byc kicha
    column = np.array(column, dtype = 'int16')
    list_of_samples.append(column)

/usr/lib/python3/dist-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [42]:
list_of_samples

[array([[ 13,   8],
        [-12, -48],
        [  8,  44],
        ..., 
        [  0,   0],
        [  0,   0],
        [  0,   0]], dtype=int16), array([[0, 0],
        [0, 0],
        [0, 0],
        ..., 
        [0, 0],
        [0, 0],
        [0, 0]], dtype=int16), array([[ 0,  0],
        [ 0,  0],
        [ 0,  1],
        ..., 
        [ 4, -1],
        [ 3,  0],
        [ 4, -2]], dtype=int16), array([[12,  8],
        [ 8,  8],
        [12, 13],
        ..., 
        [ 0,  0],
        [ 0,  0],
        [ 1,  0]], dtype=int16)]

In [60]:
#podstawowa ilosc wierszy to N
num_of_rows = [None] * ncol_track
for i in range(ncol_track):
    num_of_rows[i] = list_of_samples[i].shape[0]
max_of_rows = max(num_of_rows)

for i in range(ncol_track):
    n = list_of_samples[i].shape[0]
    if n != max_of_rows:
        list_of_samples[i] = np.vstack((list_of_samples[i], np.zeros((max_of_rows - n, 2))))

In [61]:
list_of_samples

[array([[ 13.,   8.],
        [-12., -48.],
        [  8.,  44.],
        ..., 
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]]), array([[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        ..., 
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]]), array([[ 0,  0],
        [ 0,  0],
        [ 0,  1],
        ..., 
        [ 4, -1],
        [ 3,  0],
        [ 4, -2]], dtype=int16), array([[ 12.,   8.],
        [  8.,   8.],
        [ 12.,  13.],
        ..., 
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]])]

In [64]:
sample_all = np.zeros((max_of_rows, 2))
for i in range(len(list_of_samples)):
    sample_all += 1/ncol_track * list_of_samples[i]

In [67]:
sample_all = np.array(sample_all, dtype = 'int16')

In [68]:
wav.write('wygibasy/track1.wav', 44100, sample_all)

In [72]:
!!aplay 'wygibasy/track1.wav'

["Playing WAVE 'wygibasy/track1.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo"]